## Dependencies

In [2]:
!nvidia-smi
!jupyter notebook list
%env CUDA_VISIBLE_DEVICES=3

Wed Nov  6 18:21:34 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 410.104      Driver Version: 410.104      CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   54C    P0    99W / 149W |   9115MiB / 11441MiB |     60%      Default |
+-------------------------------+----------------------+----------------------+
|   1  Tesla K80           Off  | 00000000:00:05.0 Off |                    0 |
| N/A   73C    P0    95W / 149W |   6011MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   2  T

In [3]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import time
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

from models import tiramisu
from models import tiramisu_bilinear
from models import tiramisu_m3
from models import unet
from datasets import deepglobe
from datasets import maroads
from datasets import joint_transforms
import utils.imgs
import utils.training as train_utils

# tensorboard
from torch.utils.tensorboard import SummaryWriter

## Dataset

Download the DeepGlobe dataset from https://competitions.codalab.org/competitions/18467. Place it in datasets/deepglobe/dataset/train,test,valid
Download the Massachusetts Road Dataset from https://www.cs.toronto.edu/~vmnih/data/. Combine the training, validation, and test sets, process with `crop_dataset.ipynb` and place the output in datasets/maroads/dataset/map,sat

In [4]:
run = "expM.3.drop2.1"
DEEPGLOBE_PATH = Path('datasets/', 'deepglobe/dataset')
MAROADS_PATH = Path('datasets/', 'maroads/dataset')
RESULTS_PATH = Path('.results/')
WEIGHTS_PATH = Path('.weights/')
RUNS_PATH    = Path('.runs/')
RESULTS_PATH.mkdir(exist_ok=True)
WEIGHTS_PATH.mkdir(exist_ok=True)
RUNS_PATH.mkdir(exist_ok=True)

batch_size = 1 # TODO: Should be `MAX_BATCH_PER_CARD * torch.cuda.device_count()` (which in this case is 1 assuming max of 1 batch per card)

In [5]:
# resize = joint_transforms.JointRandomCrop((300, 300))

normalize = transforms.Normalize(mean=deepglobe.mean, std=deepglobe.std)
train_joint_transformer = transforms.Compose([
#     resize,
    joint_transforms.JointRandomHorizontalFlip(),
    joint_transforms.JointRandomVerticalFlip(),
    joint_transforms.JointRandomRotate()
    ])

train_slice = slice(None,4000)
test_slice = slice(4000,None)

train_dset = deepglobe.DeepGlobe(DEEPGLOBE_PATH, 'train', slc = train_slice,
    joint_transform=train_joint_transformer,
    transform=transforms.Compose([
        transforms.ColorJitter(brightness=.4,contrast=.4,saturation=.4),
        transforms.ToTensor(),
        normalize,
    ]))

train_dset_ma = maroads.MARoads(MAROADS_PATH, 
    joint_transform=train_joint_transformer,
    transform=transforms.Compose([
        transforms.ColorJitter(brightness=.4,contrast=.4,saturation=.4),
        transforms.ToTensor(),
        normalize,
    ]))

# print(len(train_dset_ma.imgs))
# print(len(train_dset_ma.msks))
train_dset_combine = torch.utils.data.ConcatDataset((train_dset, train_dset_ma))

# train_loader = torch.utils.data.DataLoader(train_dset, batch_size=batch_size, shuffle=True)
# train_loader = torch.utils.data.DataLoader(train_dset_ma, batch_size=batch_size, shuffle=True)
train_loader = torch.utils.data.DataLoader(
    train_dset_combine, batch_size=batch_size, shuffle=True)





# resize_joint_transformer = transforms.Compose([
#     resize
#     ])
resize_joint_transformer = None
val_dset = deepglobe.DeepGlobe(
    DEEPGLOBE_PATH, 'valid', joint_transform=resize_joint_transformer,
    transform=transforms.Compose([
        transforms.ToTensor(),
        normalize
    ]))
val_loader = torch.utils.data.DataLoader(
    val_dset, batch_size=batch_size, shuffle=False)

test_dset = deepglobe.DeepGlobe(
    DEEPGLOBE_PATH, 'train', joint_transform=resize_joint_transformer, slc = test_slice,
    transform=transforms.Compose([
        transforms.ToTensor(),
        normalize
    ]))
test_loader = torch.utils.data.DataLoader(
    test_dset, batch_size=batch_size, shuffle=False)

In [83]:
print("Train: %d" %len(train_loader.dataset))
print("Val: %d" %len(val_loader.dataset.imgs))
print("Test: %d" %len(test_loader.dataset.imgs))
# print("Classes: %d" % len(train_loader.dataset.classes))

print((iter(train_loader)))

inputs, targets = next(iter(train_loader))
print("Inputs: ", inputs.size())
print("Targets: ", targets.size())

# utils.imgs.view_image(inputs[0])
# utils.imgs.view_image(targets[0])
# utils.imgs.view_annotated(targets[0])

# print(targets[0])



for i,(image,label) in enumerate(iter(test_loader)):
    if i % 10 == 0:
        print("Procssing image",i)
        
    im = image[0]
    
    # scale to [0,1]
    im -= im.min()
    im /= im.max()
    
    im = torchvision.transforms.ToPILImage()(im)
    im.save("ds_test/" + str(i) + ".png")
    
    label = label.float()
    la = torchvision.transforms.ToPILImage()(label)
    la.save("ds_test/" + str(i) + ".mask.png")
    
print("Done!")

Train: 4909
Val: 1243
Test: 2226
Inputs:  torch.Size([1, 3, 1024, 1024])
Targets:  torch.Size([1, 1024, 1024])
Procssing image 0
Procssing image 10
Procssing image 20
Procssing image 30
Procssing image 40
Procssing image 50
Procssing image 60
Procssing image 70
Procssing image 80
Procssing image 90
Procssing image 100
Procssing image 110
Procssing image 120
Procssing image 130
Procssing image 140
Procssing image 150
Procssing image 160
Procssing image 170
Procssing image 180
Procssing image 190
Procssing image 200
Procssing image 210
Procssing image 220
Procssing image 230
Procssing image 240
Procssing image 250
Procssing image 260
Procssing image 270
Procssing image 350
Procssing image 360
Procssing image 370
Procssing image 380
Procssing image 390
Procssing image 400
Procssing image 410
Procssing image 420
Procssing image 430
Procssing image 440
Procssing image 450
Procssing image 460
Procssing image 470
Procssing image 480
Procssing image 490
Procssing image 500
Procssing image 510
